### geo_agent

**geopy.geocoders**

- It simplifies the process of converting addresses into geographic coordinates (latitude and longitude) and vice versa.

In [2]:
# Example 
from geopy.geocoders import Nominatim

# Initialize the geocoder with a user agent
geolocator = Nominatim(user_agent="my_app")

# Geocode an address
location = geolocator.geocode("175 5th Avenue NYC")

# Print results
print(location.address) # Full address
print((location.latitude, location.longitude)) # Coordinates
print(location.raw) # Raw data from the geocoding service

Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan Community Board 5, Manhattan, New York County, City of New York, New York, 10010, United States of America
(40.7410592, -73.9896416)
{'place_id': 331346792, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 264768896, 'lat': '40.7410592', 'lon': '-73.9896416', 'class': 'tourism', 'type': 'attraction', 'place_rank': 30, 'importance': 9.175936522464359e-05, 'addresstype': 'tourism', 'name': 'Flatiron Building', 'display_name': 'Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan Community Board 5, Manhattan, New York County, City of New York, New York, 10010, United States of America', 'boundingbox': ['40.7407596', '40.7413004', '-73.9898715', '-73.9895014']}


In [3]:
# agents/geo_agent.py
from geopy.geocoders import Nominatim

def get_aoi(city_name: str):
    """
    Uses OpenStreetMap Nominatim API to get bounding box (AOI) for a city.
    Args:
        city_name: Name of the city to get location and aoi_box 
    """
    geolocator = Nominatim(user_agent="data_agent")
    location = geolocator.geocode(city_name)
    if not location:
        raise ValueError(f"City '{city_name}' not found.")

    # Simulated bounding box for simplicity
    bbox = {
        "min_lon": location.longitude - 0.2,
        "min_lat": location.latitude - 0.2,
        "max_lon": location.longitude + 0.2,
        "max_lat": location.latitude + 0.2,
    }

    return {
        "city": city_name,
        "lat": location.latitude,
        "lon": location.longitude,
        "aoi_bbox": bbox
    }


In [ ]:
get_aoi(city_name='')

{'city': 'Vijayawada',
 'lat': 16.5115306,
 'lon': 80.6160469,
 'aoi_bbox': {'min_lon': 80.4160469,
  'min_lat': 16.3115306,
  'max_lon': 80.8160469,
  'max_lat': 16.7115306}}

**rasterio**  
- A Python library designed for reading, writing, and analyzing geospatial raster data.

In [6]:
# agents/data_agent.py
import os
import json
from datetime import datetime
from pathlib import Path

# Simulated libraries for raster handling
import numpy as np
import rasterio
from rasterio.transform import from_origin



def fetch_land_cover(aoi, year):
    """Simulate fetching Land Cover raster"""
    print(f"[INFO] Fetching Land Cover for {aoi['city']} ({year}) ...")
    path = f"outputs/{aoi['city'].lower()}_landcover_{year}.tif"
    create_dummy_raster(path, value_range=(0, 100))
    return path


def fetch_tree_cover(aoi, year):
    """Simulate fetching Tree Canopy raster"""
    print(f"[INFO] Fetching Tree Cover for {aoi['city']} ({year}) ...")
    path = f"outputs/{aoi['city'].lower()}_treecover_{year}.tif"
    create_dummy_raster(path, value_range=(0, 100))
    return path


def fetch_ndvi(aoi, year):
    """Simulate fetching NDVI raster"""
    print(f"[INFO] Computing NDVI composite for {aoi['city']} ({year}) ...")
    path = f"outputs/{aoi['city'].lower()}_ndvi_{year}.tif"
    create_dummy_raster(path, value_range=(-1, 1))
    return path


def fetch_population(aoi, year):
    """Simulate fetching Population raster"""
    print(f"[INFO] Fetching Population raster for {aoi['city']} ({year}) ...")
    path = f"outputs/{aoi['city'].lower()}_population_{year}.tif"
    create_dummy_raster(path, value_range=(0, 500))
    return path


def create_dummy_raster(file_path, value_range):
    """Creates a small dummy GeoTIFF raster file"""
    Path("outputs").mkdir(exist_ok=True)
    data = np.random.uniform(value_range[0], value_range[1], (100, 100)).astype("float32")
    transform = from_origin(-87.9, 42.0, 0.001, 0.001)

    with rasterio.open(
        file_path,
        "w",
        driver="GTiff",
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs="EPSG:5070",
        transform=transform,
    ) as dst:
        dst.write(data, 1)


def validate_raster(file_path):
    """Check raster metadata for CRS and value ranges"""
    with rasterio.open(file_path) as src:
        stats = {
            "crs": src.crs.to_string(),
            "bounds": src.bounds,
            "min": float(src.read(1).min()),
            "max": float(src.read(1).max())
        }
    return stats


def run_data_agent(city: str, year: int, data_types=None):
    """
    Main orchestration function for Data Agent.
    """
    if data_types is None:
        data_types = ["land_cover", "tree_cover", "ndvi", "population"]

    aoi = get_aoi(city)
    job_id = f"data_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

    outputs = {}
    datasets = {}

    if "land_cover" in data_types:
        outputs["landcover_tif"] = fetch_land_cover(aoi, year)
        datasets["land_cover"] = {
            "source": "USGS/NLCD_2020_Land_Cover",
            "resolution": "30m"
        }

    if "tree_cover" in data_types:
        outputs["treecover_tif"] = fetch_tree_cover(aoi, year)
        datasets["tree_cover"] = {
            "source": "USGS/NLCD_2020_Tree_Canopy",
            "resolution": "30m"
        }

    if "ndvi" in data_types:
        outputs["ndvi_tif"] = fetch_ndvi(aoi, year)
        datasets["ndvi"] = {
            "source": "Sentinel-2 Composite via Earth Engine",
            "period": f"{year}-06-01 to {year}-08-31"
        }

    if "population" in data_types:
        outputs["population_tif"] = fetch_population(aoi, year)
        datasets["population"] = {
            "source": "WorldPop/USA_2025",
            "resolution": "100m"
        }

    # Validation example
    validation = {k: validate_raster(v) for k, v in outputs.items()}

    manifest = {
        "job_id": job_id,
        "location": city,
        "aoi_bbox": aoi["aoi_bbox"],
        "datasets": datasets,
        "validation": validation,
        "timestamp": datetime.now().isoformat()
    }

    manifest_path = f"outputs/{city.lower()}_manifest.json"
    with open(manifest_path, "w") as f:
        json.dump(manifest, f, indent=2)

    print(f"[INFO] Data Agent completed for {city}. Manifest saved to {manifest_path}")
    return {"job_id": job_id, "outputs": outputs, "manifest": manifest_path}


In [7]:
run_data_agent(city='seattle',year=2025)

[INFO] Fetching Land Cover for seattle (2025) ...
[INFO] Fetching Tree Cover for seattle (2025) ...
[INFO] Computing NDVI composite for seattle (2025) ...
[INFO] Fetching Population raster for seattle (2025) ...
[INFO] Data Agent completed for seattle. Manifest saved to outputs/seattle_manifest.json


{'job_id': 'data_20251031_214414',
 'outputs': {'landcover_tif': 'outputs/seattle_landcover_2025.tif',
  'treecover_tif': 'outputs/seattle_treecover_2025.tif',
  'ndvi_tif': 'outputs/seattle_ndvi_2025.tif',
  'population_tif': 'outputs/seattle_population_2025.tif'},
 'manifest': 'outputs/seattle_manifest.json'}